In [1]:
from utils.Variable_setup import *
from utils.Extraction_functions import *
from utils.ICC_functions import *
import os

200803-10:38:13,34 nipype.utils INFO:
	 Running nipype version 1.5.0-rc1 (latest: 1.5.0)


In [2]:
import pickle


with open(os.path.join(data_path,'SAGAB_Basic_Metric_ROIWISE_Extracted_Data_Pedro_ROIS.pckl'), 'rb') as handle:
    Extracted_Data = pickle.load(handle)

In [3]:
preprocs = ['GDT', 'Regular']
paradigms = ['Single_Band', 'Multi_Band', 'Multi_Shell']
metrics=Metrics

In [7]:
ROIS=Pedro_ROIS
ROIs_path = Pedro_ROIs_path

## Generate ICCs

In [8]:
import numpy as np
ICC_Data = {}
ICC_Data['Description'] = ' ##SAGAB### ICC Values calculated from extracted Basic Metric data'

for proc in preprocs:
    ICC_Data[proc] = {}
    for digm in paradigms:
        ICC_Data[proc][digm] = {}
        for met in Metrics:
            ICC_Data[proc][digm][met] = {}
            for ROI in ROIS:
                ICC_Data[proc][digm][met][ROI] = {}

In [11]:
ICC_Data[proc]['Multi_Band'][met][ROI]

(0.7778214194582249, 0.040908932614684736, 0.13907461919979575)

In [10]:
for ROI in ROIS:
    for proc in preprocs:
        for digm in paradigms:
            for met in Metrics:
                ICC_Data[proc][digm][met][ROI] = ICC(Extracted_Data[proc][digm][met][ROI])

In [12]:
#Save ICC numpy
with open(os.path.join(data_path,'SAGAB_Basic_Metric_PEDRO_ROIWISE_ICC_Scores.pckl'), 'wb') as handle:
    pickle.dump(Extracted_Data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Calculate Power

In [13]:
import matplotlib.pyplot as plt
import math
from statsmodels.stats.power import  tt_ind_solve_power
%matplotlib inline

Ns = [1.02, 1.04, 1.06, 1.08, 1.10]

Required_Subs_Data = {}
Required_Subs_Data['Description'] = 'Number of subjects required to detect metric change of 2/4/6/8/10%'

for ROI in ROIS:
    Required_Subs_Data[ROI] = {}
    for proc in preprocs:
        Required_Subs_Data[ROI][proc] = {}
        for digm in paradigms:
            Required_Subs_Data[ROI][proc][digm] = {}
            for met in metrics:
                Required_Subs_Data[ROI][proc][digm][met] = []
                CVbs = ICC_Data[proc][digm][met][ROI][2]
                #Required_Subs_Data[ROI][proc][digm][met] = []

                #A population of healthies has a gaussian profile with this mu, sigma 
                mu = np.nanmean(Extracted_Data[proc][digm][met][ROI])
                sigma = CVbs * mu
                for N in Ns:
                    std_diff = (mu*N - mu) / sigma
                    subjects = tt_ind_solve_power(effect_size = std_diff, nobs1 = None, alpha = 0.05, power = 0.80, ratio=1, alternative = 'larger' )
                    Required_Subs_Data[ROI][proc][digm][met].append(math.ceil(subjects))

In [14]:
import pickle


with open(os.path.join(data_path,'SAGAB_Basic_Metric_PEDRO_ROIWISE_Power_Subject_Counts.pckl'), 'wb') as handle:
    pickle.dump(Extracted_Data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
ROIS

['mask_FA_Arcuate_SAGAB_Template_Space.nii.gz',
 'mask_FA_CC_Genu_SAGAB_Template_Space.nii.gz',
 'mask_FA_CC_splenium_SAGAB_Template_Space.nii.gz',
 'mask_FA_LeftIC_SAGAB_Template_Space.nii.gz',
 'mask_FA_LFWM_SAGAB_Template_Space.nii.gz',
 'mask_FA_Orbito_Frontal_WM_SAGAB_Template_Space.nii.gz']

In [35]:
Extracted_Data['GDT']['Multi_Shell']['Axial']['Left_Caudate_OUT_GDT_ero.nii'][13,1]

819.238147

In [17]:
Required_Subs_Data['mask_FA_Arcuate_SAGAB_Template_Space.nii.gz']['GDT']

{'Single_Band': {'Axial': [69, 18, 9, 6, 4],
  'Fractional_Anisotropy': [218, 55, 25, 15, 10],
  'Radial': [366, 92, 42, 24, 16],
  'Trace': [155, 40, 18, 11, 7]},
 'Multi_Band': {'Axial': [100, 26, 12, 7, 5],
  'Fractional_Anisotropy': [164, 42, 19, 11, 8],
  'Radial': [241, 61, 28, 16, 11],
  'Trace': [110, 28, 13, 8, 6]},
 'Multi_Shell': {'Axial': [84, 22, 10, 6, 5],
  'Fractional_Anisotropy': [226, 57, 26, 15, 10],
  'Radial': [458, 115, 52, 30, 19],
  'Trace': [176, 45, 21, 12, 8]}}

In [11]:
Required_Subs_Data['Left_Caudate_OUT_GDT_ero.nii']['GDT']

{'Single_Band': {'Axial': [100, 26, 12, 7, 5],
  'Fractional_Anisotropy': [1263, 317, 141, 80, 52],
  'Radial': [102, 26, 12, 8, 5],
  'Trace': [93, 24, 11, 7, 5]},
 'Multi_Band': {'Axial': [223, 57, 26, 15, 10],
  'Fractional_Anisotropy': [792, 199, 89, 51, 33],
  'Radial': [186, 47, 22, 13, 9],
  'Trace': [196, 50, 23, 13, 9]},
 'Multi_Shell': {'Axial': [115, 30, 14, 8, 6],
  'Fractional_Anisotropy': [638, 160, 72, 41, 27],
  'Radial': [94, 24, 12, 7, 5],
  'Trace': [99, 26, 12, 7, 5]}}

In [9]:
import matplotlib.pyplot as plt
import math
from statsmodels.stats.power import  tt_ind_solve_power
%matplotlib inline

Ns = [1.01, 1.03, 1.06, 1.1, 1.15, 1.20]


proc = 'Regular'
metrics = ['Radial', 'Axial', 'Trace']
ROI_Names = ['Left_Caudate', 'Right_Caudate']

N_subs = {}
N_subs['Percentages'] = Ns

for met in metrics:
    N_subs[met] = {}
    for digm in paradigms:
        N_subs[met][digm] = {}
        for roi_i, ROI in enumerate(Caudate_ROIS):
            N_subs[met][digm][ROI_Names[roi_i]]= []

for met in metrics:
    for digm in paradigms:
        for roi_i, ROI in enumerate(Caudate_ROIS):
            CVbs = ICC_Data[proc][digm][met][ROI][2]

            #A population of healthies has a gaussian profile with this mu, sigma 
            mu = np.nanmean(Extracted_Data[proc][digm][met][ROI])
            sigma = CVbs * mu
            for N in Ns:
                std_diff = (mu*N - mu) / sigma
                N_subs[met][digm][ROI_Names[roi_i]].append(math.ceil(tt_ind_solve_power(effect_size = std_diff, nobs1 = None, alpha = 0.05, power = 0.80, ratio=1, alternative = 'larger' )))
                

IndexError: list index out of range

In [10]:
Required_Subs_Data['Left_Caudate_OUT_GDT_ero.nii']['GDT']

{'Single_Band': {'Axial': [100, 26, 12, 7, 5],
  'Fractional_Anisotropy': [1263, 317, 141, 80, 52],
  'Radial': [102, 26, 12, 8, 5],
  'Trace': [93, 24, 11, 7, 5]},
 'Multi_Band': {'Axial': [223, 57, 26, 15, 10],
  'Fractional_Anisotropy': [792, 199, 89, 51, 33],
  'Radial': [186, 47, 22, 13, 9],
  'Trace': [196, 50, 23, 13, 9]},
 'Multi_Shell': {'Axial': [115, 30, 14, 8, 6],
  'Fractional_Anisotropy': [638, 160, 72, 41, 27],
  'Radial': [94, 24, 12, 7, 5],
  'Trace': [99, 26, 12, 7, 5]}}